## LSTM 모델 활용 
conda activate tf2

#### 데이터 로드 및 처리 

In [1]:
import numpy as np
import pandas as pd 
import datetime
import matplotlib.pyplot as plt 
%matplotlib inline 

plt.rc('font', family = 'AppleGothic')
plt.rcParams['axes.unicode_minus'] = False


df = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding = 'euc-kr')
test = pd.read_csv('가스공급량 수요예측모델개발_data/test.csv')
sub = pd.read_csv('가스공급량 수요예측모델개발_data/sample_submission.csv')

def tohour(val):
    if len(val) == 1:
        return '0'+val+':00:00'
    else:
        return val+':00:00'
    
def gubun(df):
    df = df[['연월일시간','공급량']]
    df.columns = ['ds','y']
    df = df.set_index('ds')
    return df

In [2]:
df['시간_수정'] = df['시간'] -1 
df['시간_수정'] = df['시간_수정'].astype(str)
df['시간_수정'] = df['시간_수정'].apply(tohour)
df['연월일시간'] = df['연월일']+' '+df['시간_수정']
df['연월일시간'] = pd.to_datetime(df['연월일시간'])

In [3]:
A = df[df['구분'] =='A']
B = df[df['구분'] =='B']
C = df[df['구분'] =='C']
D = df[df['구분'] =='D']
E = df[df['구분'] =='E']
G = df[df['구분'] =='G']
H = df[df['구분'] =='H']
a = gubun(A)
b = gubun(B)
c = gubun(C)
d = gubun(D)
e = gubun(E)
g = gubun(G)
h = gubun(H)

### 모델링

#### 데이터 스케일링 및 분할 

In [4]:
def split_df(df, times):
    x_train = []
    y_train = []
    df = np.array(df)
    for i in range(len(df) - times):
        x = df[i:i+times]
        y = df[i+times:i+times + times]
        
        if len(y) == times :
            x_train.append(x)
            y_train.append(y)
        else :
            break
        
    
    return x_train, y_train

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
## 데이터 scaling 
scaler_a = MinMaxScaler(feature_range = (0, 1))
scaler_b = MinMaxScaler(feature_range = (0, 1))
scaler_c = MinMaxScaler(feature_range = (0, 1))
scaler_d = MinMaxScaler(feature_range = (0, 1))
scaler_e = MinMaxScaler(feature_range = (0, 1))
scaler_g = MinMaxScaler(feature_range = (0, 1))
scaler_h = MinMaxScaler(feature_range = (0, 1))

In [7]:
sc_a = scaler_a.fit_transform(a)
sc_b = scaler_b.fit_transform(b)
sc_c = scaler_c.fit_transform(c)
sc_d = scaler_d.fit_transform(d)
sc_e = scaler_e.fit_transform(e)
sc_g = scaler_g.fit_transform(g)
sc_h = scaler_h.fit_transform(h)

In [18]:
a.isna().sum()

y    0
dtype: int64

In [25]:
h.isna().sum()

y    0
dtype: int64

In [8]:
def prep_df(sc_df):
    x_train, y_train = split_df(sc_df, 2160)
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    
    return x_train, y_train

#### LSTM 모델링

In [9]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, Dropout
from tensorflow.keras.models import Model, Sequential

In [10]:
tf.__version__

'2.6.0'

In [41]:
def modeling(batch_size):
    
    model = Sequential()
    # stateful = True 이전 배치의 학습 상태가 다음 배치 작업에 영향 
    model.add(LSTM(32, batch_input_shape = (batch_size,2160,1), return_sequences = True)) 
    model.add(Dropout(0.2))
    model.add(LSTM(32, batch_input_shape = (batch_size,2160,1), return_sequences = True)) 
    model.add(Dropout(0.2))
    model.add(TimeDistributed(Dense(1)))
    
    model.summary()
    model.compile(loss ='mae', optimizer = 'adam')
    
    return model

In [42]:
batch_size = 1000
model = modeling(batch_size)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (1000, 2160, 32)          4352      
_________________________________________________________________
dropout_8 (Dropout)          (1000, 2160, 32)          0         
_________________________________________________________________
lstm_9 (LSTM)                (1000, 2160, 32)          8320      
_________________________________________________________________
dropout_9 (Dropout)          (1000, 2160, 32)          0         
_________________________________________________________________
time_distributed_4 (TimeDist (1000, 2160, 1)           33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


In [43]:
x_train, y_train = prep_df(sc_a)

In [44]:
train_x = x_train[:int(len(x_train)*0.8)]
val_x = x_train[int(len(x_train)*0.8):]

train_y = y_train[:int(len(x_train)*0.8)]
val_y = y_train[int(len(x_train)*0.8):]

In [45]:
model.fit(train_x, train_y,batch_size = batch_size , epochs = 10,
         validation_data = (val_x, val_y))

Epoch 1/10
38/39 [============================>.] - ETA: 7s - loss: 0.0505 

InvalidArgumentError:    Specified a list with shape [1000,1] from a tensor with shape [612,1]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[sequential_4/lstm_8/PartitionedCall]] [Op:__inference_train_function_23429]

Function call stack:
train_function -> train_function -> train_function


In [ ]:
bx_train, by_train = prep_df(sc_b)
b_model = modeling(batch_size)
b_model.fit(bx_train, by_train, batch_size = batch_size, epochs = 10)

In [ ]:
cx_train, cy_train = prep_df(sc_c)
c_model = modeling(batch_size)
c_model.fit(cx_train, cy_train, batch_size = batch_size, epochs = 10)

In [ ]:
dx_train, dy_train = prep_df(sc_d)
d_model = modeling(batch_size)
d_model.fit(dx_train, dy_train, batch_size = batch_size, epochs = 10)

In [ ]:
ex_train, ey_train = prep_df(sc_e)
e_model = modeling(batch_size)
e_model.fit(ex_train, ey_train, batch_size = batch_size, epochs = 10)

In [ ]:
gx_train, gy_train = prep_df(sc_g)
g_model = modeling(batch_size)
g_model.fit(gx_train, gy_train, batch_size = batch_size, epochs = 100)

In [ ]:
hx_train, hy_train = prep_df(sc_h)
h_model = modeling(batch_size)
h_model.fit(hx_train, hy_train, batch_size = batch_size, epochs = 100)